# Check Neural Collapse on the Input Embedding

In [1]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from types import SimpleNamespace
from typing import Dict, Any

import _pickle as pickle
import argparse
import copy
import jax
import numpy as np
import os
import timeit

from jaxl.constants import *
from jaxl.datasets import get_dataset
from jaxl.models import load_config, iterate_models, get_model
from jaxl.utils import parse_dict, get_device


from utils import *

/Users/chanb/research/personal/icl/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
device = "cpu"
# device = "gpu:0"
get_device(device)

In [3]:
learner_path = "/Users/chanb/research/personal/icl/saved_models/icl-omniglot/async-04-16-24_12_53_49-019260d2-241d-4285-b98e-d1d9fe05e929"

In [4]:
config_dict, config = load_config(learner_path)

if config.learner_config.dataset_config.dataset_kwargs.save_path.startswith("/home/bryanpu1"):
    config_dict["learner_config"]["dataset_config"]["dataset_kwargs"]["save_path"] = "/Users/chanb/research/personal/icl/data"
    config_dict["learner_config"]["dataset_config"]["dataset_kwargs"]["task_config"]["save_dir"] = "/Users/chanb/research/personal/icl/data/omniglot_icl"
    config_dict["learner_config"]["dataset_config"]["dataset_kwargs"]["task_config"]["num_sequences"] = 320
    config_dict["learner_config"]["dataset_config"]["num_workers"] = 0
    config = parse_dict(config_dict)

train_dataset = get_dataset(
    config.learner_config.dataset_config,
    config.learner_config.seeds.data_seed,
)

context_len = config.model_config.num_contexts

fixed_length = True
if hasattr(config.learner_config.dataset_config, "dataset_wrapper"):
    fixed_length = config.learner_config.dataset_config.dataset_wrapper.type in [
        "FixedLengthContextDataset"
    ]

Files already downloaded and verified
Files already downloaded and verified
Loading from /Users/chanb/research/personal/icl/data/omniglot_icl/omniglot_bursty-all_split-p_bursty_1.0-num_sequences_320-sequence_length_9-min_num_per_class_1-random_label_False-seed_44.pkl


In [5]:
checkpoint_steps = []
all_tokens = []
for params, model, checkpoint_step in iterate_models(
    train_dataset.input_dim, train_dataset.output_dim, learner_path
):
    dataset_loader = train_dataset.get_dataloader(config.learner_config)
    checkpoint_steps.append(checkpoint_step)
    for batch_i, data in enumerate(dataset_loader):
        context_inputs = data["context_inputs"]
        context_outputs = data["context_outputs"]
        queries = data["queries"]
        one_hot_labels = data["outputs"]

        if hasattr(context_inputs, "numpy"):
            context_inputs = context_inputs.numpy()
            context_outputs = context_outputs.numpy()
            queries = queries.numpy()
            one_hot_labels = one_hot_labels.numpy()

        tokens, _, _ = model.tokenize(
            params[CONST_MODEL_DICT][CONST_MODEL],
            queries,
            {
                CONST_CONTEXT_INPUT: context_inputs,
                CONST_CONTEXT_OUTPUT: context_outputs,
            },
            eval=True,
        )
        all_tokens.append(tokens)

all_tokens = np.concatenate(all_tokens, axis=0)

/Users/chanb/research/personal/icl/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/chanb/research/personal/icl/jaxl/jaxl/datasets/wrappers.py", line 19, in __getattr__
    return getattr(self._dataset, attr)
  File "/Users/chanb/research/personal/icl/jaxl/jaxl/datasets/wrappers.py", line 19, in __getattr__
 

KeyboardInterrupt: 

/Users/chanb/research/personal/icl/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
all_tokens.shape